In [32]:
import numpy as np
import pandas as pd
import pyspark
from pyspark.mllib.recommendation import ALS
import math
import matplotlib.pyplot as plt
% matplotlib inline

In [14]:
sc = pyspark.SparkContext(master = "local[3]")

In [1]:
# link to move data
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

In [8]:
import os

datasets_path = os.path.join('/Users/Alexander/WINE/', 'data')

small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')

In [9]:
import urllib

small_f = urllib.urlretrieve (small_dataset_url, small_dataset_path)

In [10]:
import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)

In [15]:
small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')

small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]

In [16]:
small_ratings_raw_data_header

u'userId,movieId,rating,timestamp'

In [17]:
small_ratings_raw_data.take(2)

[u'userId,movieId,rating,timestamp', u'1,16,4.0,1217897793']

In [18]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
                                           .map(lambda line: line.split(","))\
                                           .map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [19]:
# user_id, movie_id, rating
small_ratings_data.take(2)

[(u'1', u'16', u'4.0'), (u'1', u'24', u'1.5')]

In [21]:
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')

small_movies_raw_data = sc.textFile(small_movies_file)
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()

In [22]:
# movie_id, title  -- genre has been droped
small_movies_data.take(3)

[(u'1', u'Toy Story (1995)'),
 (u'2', u'Jumanji (1995)'),
 (u'3', u'Grumpier Old Men (1995)')]

In [23]:
# split data into train and test sets 
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [31]:
# user_id, movie_id, rating
training_RDD.take(1)

[(u'1', u'24', u'1.5')]

In [29]:
validation_RDD.take(1)

[(u'1', u'47', u'4.0')]

In [30]:
test_RDD.take(1)

[(u'1', u'16', u'4.0')]

In [35]:
seed = 5L
# iterations through optimization
iterations = 10
regularization_parameter = 0.1
# explore different ranks (n_latent feats) to find best performing model
ranks = [4, 8, 12]
# save RMSE for each rank
errors = [0, 0, 0]
# error counter
err = 0
# IDK
tolerance = 0.02

# IDK
min_error = float('inf')
# stores int of best rank
best_rank = -1
# IDK
best_iteration = -1

# iterate through different ranks (n_latent feats) to find best performing model
for rank in ranks:
    
    model = ALS.train(training_RDD, 
                      rank, 
                      seed=seed, 
                      iterations=iterations,
                      lambda_=regularization_parameter)
    
    # ((r[0], r[1]), r[2]) --> user_id, movie_id, rating
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    
    # error = (y_pred - y_true)**2
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
    errors[err] = error
    
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 0.913148264878
For rank 8 the RMSE is 0.923277139628
For rank 12 the RMSE is 0.929193078284
The best model was trained with rank 4


In [36]:
errors

[0.9131482648783014, 0.9232771396279622, 0.9291930782840704]

In [37]:
# ( (UserID, the MovieID), Pred_Rating)
predictions.take(3)

[((147, 3702), 3.816088021779933),
 ((24, 3702), 3.4055680376562143),
 ((69, 3702), 3.6241886521955657)]

In [38]:
# Then we join these with our validation data (the one that includes ratings) and the result looks as follows:
#  ( (user_id, movie_id) , (true_rating, predicted_rating)
rates_and_preds.take(3)

[((383, 1298), (3.5, 3.5951446081982064)),
 ((575, 2468), (2.0, 2.660015379461113)),
 ((528, 30749), (4.5, 4.1952308608676825))]

### Cross Validation

In [39]:
model = ALS.train(training_RDD, 
                  best_rank, 
                  seed=seed, 
                  iterations=iterations,
                  lambda_=regularization_parameter)

predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))

rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)

error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

For testing data the RMSE is 0.916442689193


In [56]:
# Parse
small_movies_data = small_ratings_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

small_movies_titles = small_ratings_data.map(lambda x: (int(x[0]),x[1]))
    
print "There are %s movies in the complete dataset" % (small_ratings_data.count())

There are 105339 movies in the complete dataset


In [57]:
def get_counts_and_averages(ID_and_ratings_tuple):
    n_ratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (n_ratings, float(sum(x for x in ID_and_ratings_tuple[1]))/n_ratings)

In [58]:
movie_ID_with_ratings_RDD = (small_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())

movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)

movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [59]:
movie_ID_with_ratings_RDD.take(1)

[(u'73469', <pyspark.resultiterable.ResultIterable at 0x10a236e90>)]

In [60]:
movie_ID_with_avg_ratings_RDD.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 969.0 failed 1 times, most recent failure: Lost task 0.0 in stage 969.0 (TID 450, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/Alexander/spark-1.6.0-bin-hadoop2.4/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/Alexander/spark-1.6.0-bin-hadoop2.4/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/Alexander/spark-1.6.0-bin-hadoop2.4/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/Alexander/spark-1.6.0-bin-hadoop2.4/python/pyspark/rdd.py", line 1293, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-57-2996849c8ec